# PROJET TVID

In [1]:
import numpy as np
from PIL import Image

## A. Jouer un flux MPEG-2 élémentaire de test

### 1. Dans le dossier videos/elementary, avec vlc, visualisez les séquences MPEG-2 suivantes : 
### _vlc -V \<renderer> bw_numbers.m2v / pendulum.m2v / Jaggies1.m2v_

Les commandes utilisees sont :
- vlc -V x11 bw_numbers.m2v
- vlc -V x11 pendulum.m2v
- vlc -V x11 Jaggies1.m2v

 <img src="./screenshots/1.png" title="bw_numbers">
 <img src="./screenshots/2.png" title="pendulum">
 <img src="./screenshots/3.png" title="Jaggles1">

### 2. Avec mpeg2dec, convertissez en pile d’images votre séquence MPEG-2 choisie (cf. aide de mpeg2dec).

Nous avons cree un dossier pgm_images a la racine du projet, et avons effectue cette commande :
- _../tools/mpeg2dec/src/mpeg2dec -o pgm ../videos/elementary/bw_numbers.m2v_

Pour convertir la sequence bw_numbers.m2v en pile d'images.

 <img src="./screenshots/pgm_0.png" title="0">

### 3. Observez les pgm générées. Comment sont-elles structurées? Quel est le format de l’image: résolution, profondeur, sampling mode?

In [2]:
img = Image.open('./pgm_images/0.pgm')

print(f"Format: {img.format}")
print(f"Resolution: {img.size}")
print(f"Mode: {img.mode}")
print(f"Color palette: {img.palette}")

Format: PPM
Resolution: (720, 720)
Mode: L
Color palette: None


Nos images sont au format PGM encodes en binaire. Ce sont des images en niveaux de gris (sur 8-bit). Elles ont une resolution de 720x720.

 <img src="./screenshots/bw_numbers_infos.png" title="bw_numbers_infos">
 
En regardant les informations de la video originale, nous pouvons voir que la resolution est de 720x480, que la profondeur est de 8-bit, et que le sampling mode est en 4:2:0 YUV.

Les frames sont composes de 2 images consécutives entrelacées dans une même image (top first field).

Les informations de la video ont ete generees par cette commande :
- ffmpeg -i bw_numbers.m2v

### 4. Modifiez mpeg2dec pour logger simplement les flags progressive_frame, top_field_first, repeat_first_field de chaque image décodée.

Pour logger simplement les flags progressive_frame (PROG), top_field_first (TFF) et repeat_first_field (RFF) pour chaque image decodee, il faut modifier le fichier :
- mpeg2dec/src/mpeg2dec.c, ligne 55

en mettant la variable __verbose__ a 1 :
- static int verbose = 1;

On obtient par exemple ces logs pour la video bw_numbers.m2v :
 <img src="./screenshots/bw_numbers_logs.png" title="bw_numbers_logs">
 
Les images decodee ici sont toutes TFF.

### 5. Avec votre propre code et dans le langage de votre choix, implémentez un convertisseur d’images vers un format plus humainement lisible (ppm RGB est assez universel).

Le langage choisi est Python.

Nous savons que nos images sont en format YUV 4:2:0. 

Cela signifie que pour un carree de 4 pixels, les 4 contiennent une information de luminance, mais seulement un contient une information de chrominance u, et un autre contient une information de chrominance v :
<img src="./screenshots/420.png" title="420">

Nous avons remarque que 2/3 de nos image PGM etait consacre a donne l'information de luminance, un sixieme etait consacre a donne l'information de chrominance u, et un autre sixieme etait consacre a donne l'information de chrominance v. 

Nous avons alors isole les differentes parties de l'image en fonction du type d'information qu'elles conviaient.

Nous avons ensuite elargi nos arrays de chrominances en dupliquant les valeurs en largeur et en hauteur pour matcher la shape de l'array de luminance, avant de les concatener en un seul array.

Nous avons enfin normalise et converti nos valeurs YUV en RGB par une relation trouvee sur la page wikipedia sur le format YUV :
<img src="./screenshots/normalization.png" title="normalization">
<img src="./screenshots/conversion_formula.png" title="conversion_formula">
 
La conversion de PGM YUV 4:2:0 en PPM RGB est alors naturel.

In [10]:
def YUV_to_RGB(y, u, v) :
    r = y + 1.13983 * v
    g = y - 0.39465 * u - 0.58060 * v
    b = y + 2.03211 * u
    
    return r, g, b

In [11]:
def PGM_to_PPM(yuv_array, TFF=True) :
    # loading pgm info

    height, width = yuv_array.shape
    
    uv_width = width // 2
    y_height = int(height * 2 / 3)
    
    # extraction
    
    y_array = yuv_array[0:y_height, 0:width]
    u_array = yuv_array[y_height:height, 0:uv_width]
    v_array = yuv_array[y_height:height, uv_width:width]
    
    # duplication (to match the size of y)
    
    u_array = np.repeat(np.repeat(u_array, 2, axis=1), 2, axis=0)
    v_array = np.repeat(np.repeat(v_array, 2, axis=1), 2, axis=0)
    
    if TFF :
        y_array = np.repeat(y_array, 2, axis=0)
        u_array = np.repeat(u_array, 2, axis=0)
        v_array = np.repeat(v_array, 2, axis=0)
        
        y_height *= 2
    
    # normalization

    y_array = y_array / 255
    
    u_max = 0.436
    u_min = -u_max
    u_array = u_min + (u_array / 255) * 2 * u_max
    
    v_max = 0.615
    v_min = -v_max
    v_array = v_min + (v_array / 255) * 2 * v_max
    
    # stacking
    
    yuv_data = np.stack((y_array, u_array, v_array), axis=-1)
    
    # conversion to rgb data between [0, 1]
    
    rgb_data = []
    
    for i in range(y_height) :
        rgb_data.append([])
        
        for j in range(width) :
            y = yuv_data[i][j][0]
            u = yuv_data[i][j][1]
            v = yuv_data[i][j][2]
            
            r, g, b = YUV_to_RGB(y, u, v)
            
            rgb_data[i].append([r, g, b])
            
    rgb_data = np.array(rgb_data)
    
    rgb_data = np.clip(rgb_data, 0, 1)
            
    # conversion to uint-8 values between [0, 255]
    
    rgb_data = (rgb_data * 255).astype('uint8')
    
    return rgb_data

In [19]:
def generic_PGM_to_PPM(pgm_path, ppm_path=None, TFF=True) :
    # loading pgm
    
    img = Image.open(pgm_path)
    
    yuv_array = np.array(img)
    
    # top first field order
    
    if TFF :
        rgb_data_top = PGM_to_PPM(yuv_array[::2], TFF=True)
        rgb_data_bottom = PGM_to_PPM(yuv_array[1::2], TFF=True)
        
        # saving as ppm
        
        if ppm_path :
            new_img_1 = Image.fromarray(rgb_data_top)
            new_img_2 = Image.fromarray(rgb_data_bottom)
            
            new_img_1.save(ppm_path + "_1.ppm")
            new_img_2.save(ppm_path + "_2.ppm")
            
        return rgb_data_top, rgb_data_bottom
    
    rgb_data = PGM_to_PPM(yuv_array, TFF=False)
    
    # saving as ppm
    
    if ppm_path :
        new_img = Image.fromarray(rgb_data)
        new_img.save(ppm_path + ".ppm")
    
    return rgb_data

In [20]:
# Example usage
pgm_path = './pgm_images/0.pgm'
ppm_path = './ppm_images/0'
generic_PGM_to_PPM(pgm_path, ppm_path)
print("It worked !")

It worked !


 <img src="./screenshots/ppm_real_0.png" title="ppm_0">

L'image PPM est identique a celle precedemment montre en PGM, tout en recuperant l'information chromatique.

La fonction de conversion semble donc fonctionner comme attendu.

### Les questions 5 a 10 ont ete realise dans les fichiers project.py et pgm_to_ppm.py